In [1]:
#@title Kaggle API

from IPython.display import clear_output

username = str(input("username: "))
key = str(input("key: "))

clear_output()

f = open("kaggle.json", "w")
f.write('{"username":"'+username+'","key":"'+key+'"}')
f.close()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

import os, re
from zipfile import ZipFile

with ZipFile('/content/anime-recommendation-database-2020.zip', 'r') as zipObj:
   zipObj.extractall()

def purge(dir, pattern):
    for f in os.listdir(dir):
        if pattern in f:
            os.remove(os.path.join(dir, f))

purge("/content/", ".zip")
purge("/content/", ".json")

 99% 651M/661M [00:05<00:00, 164MB/s]
100% 661M/661M [00:05<00:00, 123MB/s]


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

In [15]:
interactions = pd.read_csv("/content/animelist.csv")

Let's add a new feature `interaction`, which will be a 1 if the user had a positive interaction, 0.5 if we don't know, and 0 otherwise. We will model the interaction values using the `watching_status`.

```
1: Currently Watching
2: Completed
3: On Hold
4: Dropped
6: Plan to Watch
```

First, let's remove incorrect data.

In [16]:
interactions = interactions[interactions["watching_status"] <= 4]
interactions = interactions[interactions["watching_status"] >= 2]
interactions = interactions[interactions["watching_status"] != 3]

We'll model interactions using the following function

In [17]:
def transform(df):
    df.loc[df["rating"] <= 5, ["watching_status"]] = 0.0
    df.loc[df["watching_status"] == 2, ["watching_status"]] = 1.0
    df.loc[df["watching_status"] == 4, ["watching_status"]] = 0.0

    df.rename(columns = {'watching_status': 'interaction'}, inplace = True)

In [18]:
transform(interactions)

In [29]:
good = interactions['user_id'].value_counts().loc[lambda x : x>4].unique()

In [35]:
interactions = interactions[interactions["user_id"].isin(good)]

In [37]:
interactions["interaction"].value_counts()

1    365972
0    143885
Name: interaction, dtype: int64

In [ ]:
interactions = interactions.drop(columns = ["rating", "watched_episodes"])
interactions.head(5)

In [39]:
num_users = interactions["user_id"].nunique()
num_anime = interactions["anime_id"].nunique()

print(f"There are a total of {num_users} users found.")
print(f"There are a total of {num_anime} anime found.")

print("Max user id:", interactions["user_id"].max())
print("Max anime id:", interactions["anime_id"].max())

There are a total of 2365 users found.
There are a total of 11663 anime found.
Max user id: 16819
Max anime id: 48456


If we take a look at the anime and user ids, we can notice that some values there are missing. Let's make a two helper functions, for easy conversion between dataset ids and embedding ids.

In [40]:
embId2user = sorted(interactions["user_id"].unique())
embId2anime = sorted(interactions["anime_id"].unique())

In [41]:
user2embId = {v: k for k, v in enumerate(embId2user)}
anime2embId = {v: k for k, v in enumerate(embId2anime)}

Now, let's make the dataset. It will look like a tuples of `(user_id, anime_embId, rating)`. We'll make 3 sets: `train`, `test` and `val`.

In [42]:
np.random.seed(seed = 42)

def makeDataSet(df, split=0.95):
    n = df.to_numpy()
    
    n = np.random.permutation(n)

    x = n[:, :2]
    y = n[:, 2]

    for i in range(x.shape[0]):
        x[i][0] = user2embId[x[i][0]]
        x[i][1] = anime2embId[x[i][1]]

    s1 = int(split * n.shape[0])
    s2 = s1 + int((1 - split) * n.shape[0] / 10)
    
    return (x[:s1], y[:s1], x[s1:s2], y[s1:s2], x[s2:], y[s2:])

In [44]:
x_train, y_train, x_test, y_test, x_val, y_val = makeDataSet(interactions)

# Models

In [45]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Matrix Factorization

In [46]:
class MatrixFactorizationModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(MatrixFactorizationModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.user_biases = tf.keras.layers.Embedding(num_users, 1)
        self.item_biases = tf.keras.layers.Embedding(num_items, 1)

        self.bias = tf.Variable(tf.zeros([1]))

        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids) + self.user_biases(user_ids)
        item_embedding = self.item_embeddings(item_ids) + self.item_biases(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        dot = tf.keras.layers.Dot(axes=1)([user_embedding, item_embedding]) + self.bias

        return tf.math.sigmoid(dot)
        
    def plot_model(self, 
                   to_file = 'model.png', dpi = 96, 
                   show_shapes = True, 
                   show_layer_names = True, 
                   expand_nested = True):
        x = tf.keras.layers.Input(shape=(2, 1))

        return tf.keras.utils.plot_model(
            tf.keras.Model(inputs=[x], outputs = self.call(x)),
            to_file = to_file, 
            dpi = dpi,
            show_shapes = show_shapes, 
            show_layer_names = show_layer_names,
            expand_nested = expand_nested,
            # rankdir = 'LR'  
        )

In [52]:
mf_model = MatrixFactorizationModel(num_users = num_users, 
                                    num_items = num_anime, 
                                    embedding_dim = 64)

mf_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = [
        tf.keras.metrics.BinaryAccuracy("Acc"),
        tf.keras.metrics.Precision(name = "P"),
        tf.keras.metrics.Recall(name = "R")
    ],
    run_eagerly = True
)

In [53]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = mf_model.fit(
    x = x_train, y = y_train,
    class_weight = {
        0: 2.5,
        1: 1.
    },
    batch_size = 64, 
    epochs = 100, 
    steps_per_epoch = 1000,
    callbacks = [callback],
    validation_data = (x_test, y_test),
    validation_steps = 10,
    validation_batch_size = 64
)

Epoch 1/100
1000/1000 [==============================] - 50s 50ms/step - loss: 0.9892 - Acc: 0.5313 - P: 0.7158 - R: 0.5744 - val_loss: 0.6886 - val_Acc: 0.5750 - val_P: 0.7338 - val_R: 0.6413
Epoch 2/100
1000/1000 [==============================] - 50s 50ms/step - loss: 0.9642 - Acc: 0.5844 - P: 0.7527 - R: 0.6286 - val_loss: 0.6032 - val_Acc: 0.6953 - val_P: 0.8208 - val_R: 0.7370
Epoch 3/100
1000/1000 [==============================] - 49s 49ms/step - loss: 0.8156 - Acc: 0.7141 - P: 0.8501 - R: 0.7310 - val_loss: 0.4809 - val_Acc: 0.7453 - val_P: 0.8877 - val_R: 0.7391
Epoch 4/100
1000/1000 [==============================] - 48s 48ms/step - loss: 0.7376 - Acc: 0.7450 - P: 0.8823 - R: 0.7452 - val_loss: 0.4679 - val_Acc: 0.7531 - val_P: 0.8892 - val_R: 0.7500
Epoch 5/100
1000/1000 [==============================] - 49s 49ms/step - loss: 0.7266 - Acc: 0.7493 - P: 0.8856 - R: 0.7468 - val_loss: 0.4582 - val_Acc: 0.7625 - val_P: 0.9031 - val_R: 0.7500
Epoch 6/100
1000/1000 [============

In [55]:
mf_model.save_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/Interactions/MatrixFactorizationModel/model", overwrite=True
)

In [ ]:
mf_model.load_weights(
    "/content/drive/Shareddrives/ML/Anime RecSysInteractions/MatrixFactorizationModel/model"
)

## Neural Network

In [56]:
class NeuralNetworkModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(NeuralNetworkModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        x = self.concat([user_embedding, item_embedding])
        x = self.dense1(x)
        x = self.dense2(x)

        return x
    
    def plot_model(self, 
                   to_file = 'model.png', dpi = 96, 
                   show_shapes = True, 
                   show_layer_names = True, 
                   expand_nested = True):
        x = tf.keras.layers.Input(shape=(2, 1))

        return tf.keras.utils.plot_model(
            tf.keras.Model(inputs=[x], outputs = self.call(x)),
            to_file = to_file, 
            dpi = dpi,
            show_shapes = show_shapes, 
            show_layer_names = show_layer_names,
            expand_nested = expand_nested,
            # rankdir = 'LR'  
        )

In [57]:
nn_model = NeuralNetworkModel(num_users = num_users, 
                              num_items = num_anime, 
                              embedding_dim = 64)

nn_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = [
        tf.keras.metrics.BinaryAccuracy("Acc"),
        tf.keras.metrics.Precision(name = "P"),
        tf.keras.metrics.Recall(name = "R")
    ],
    run_eagerly = True
)

In [58]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = nn_model.fit(
    x = x_train, y = y_train, 
    class_weight = {
        0: 2.5,
        1: 1.
    },
    batch_size = 64, 
    epochs = 100,
    steps_per_epoch = 1000,
    callbacks = [callback],
    validation_data = (x_test, y_test),
    validation_steps = 10,
    validation_batch_size = 64
)

Epoch 1/100
1000/1000 [==============================] - 41s 41ms/step - loss: 0.7679 - Acc: 0.7510 - P: 0.8495 - R: 0.7943 - val_loss: 0.4471 - val_Acc: 0.8109 - val_P: 0.8742 - val_R: 0.8609
Epoch 2/100
1000/1000 [==============================] - 41s 41ms/step - loss: 0.6865 - Acc: 0.7789 - P: 0.8801 - R: 0.8000 - val_loss: 0.4493 - val_Acc: 0.7969 - val_P: 0.9064 - val_R: 0.8000
Epoch 3/100
1000/1000 [==============================] - 41s 41ms/step - loss: 0.6649 - Acc: 0.7818 - P: 0.8875 - R: 0.7977 - val_loss: 0.4300 - val_Acc: 0.8078 - val_P: 0.8874 - val_R: 0.8391
Epoch 4/100
1000/1000 [==============================] - 40s 40ms/step - loss: 0.6512 - Acc: 0.7885 - P: 0.8925 - R: 0.8021 - val_loss: 0.4303 - val_Acc: 0.8156 - val_P: 0.9150 - val_R: 0.8196
Epoch 5/100
1000/1000 [==============================] - 41s 41ms/step - loss: 0.6461 - Acc: 0.7893 - P: 0.8925 - R: 0.8036 - val_loss: 0.4243 - val_Acc: 0.8156 - val_P: 0.9052 - val_R: 0.8304
Epoch 6/100
1000/1000 [============

In [59]:
nn_model.save_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/Interactions/NeuralNetworkModel/model", overwrite=True
)

In [ ]:
nn_model.load_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/Interactions/NeuralNetworkModel/model"
)

## Comparing the models

In [60]:
mf_model.evaluate(x = x_val, y = y_val)

717/717 [==============================] - 18s 26ms/step - loss: 0.4691 - Acc: 0.7678 - P: 0.8969 - R: 0.7621


[0.4691160321235657,
 0.7678260207176208,
 0.8968790173530579,
 0.7621142864227295]

In [61]:
nn_model.evaluate(x = x_val, y = y_val)

717/717 [==============================] - 17s 24ms/step - loss: 0.4211 - Acc: 0.8050 - P: 0.8907 - R: 0.8282


[0.4211376905441284,
 0.8049598932266235,
 0.8906558156013489,
 0.8282309770584106]